# Table detector

## Colab requirements

Before restarting runtime (remember to select GPU runtime)$\dots$

In [ ]:
!git clone https://github.com/Wadaboa/table-detector.git
!pip install -r table-detector/init/colab_requirements.txt

After restarting runtime$\dots$

In [ ]:
import os, sys

sys.path.insert(0, "/content/table-detector")
sys.path.insert(0, "/content/table-detector/src")
os.chdir("/content/table-detector")

## Imports

In [ ]:
import torch
import wandb
import yaml
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, fixed

import learning
import transforms
import detectors
import utils
import train

## Initialization

### Matplotlib

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 12]
plt.rcParams['figure.dpi'] = 100
plt.rcParams['axes.xmargin'] = .05
plt.rcParams['axes.ymargin'] = .05
plt.style.use('ggplot')

### Weights & biases

In [ ]:
!wandb login

In [ ]:
!wandb enabled

### PyTorch and numpy

In [ ]:
DEVICE = utils.get_device()
DEVICE

In [ ]:
if DEVICE.type != "cpu":
    !nvidia-smi

## Utils

In [ ]:
def show(img):
    np_img = utils.to_numpy(img)
    plt.imshow(np_img)

def draw_predictions(dataset, index, model, conf_thresh=0.5, width=4):
    images = []
    for image, _ in dataset[index]:
        output = model(image)
        boxes = output["boxes"][output["scores"] >= conf_thresh]
        colors = generate_colors(len(output["boxes"]))
        img = utils.draw_bounding_boxes(
            utils.denormalize_image(image), boxes, width=width, colors=colors
        )
        images.append(img)
    grid = torchvision.utils.make_grid(images)
    show(grid)

def draw_targets(dataset, index, width=4):
    images = []
    for image, target in dataset[index]:
        colors = generate_colors(len(target["boxes"]))
        img = utils.draw_bounding_boxes(
            utils.denormalize_image(image), target["boxes"], width=width, colors=colors
        )
        images.append(img)
    grid = torchvision.utils.make_grid(images)
    show(grid)

## Data loading

In [ ]:
with open('parameters.yml', 'r') as conf:
    args = yaml.load(conf, Loader=yaml.FullLoader)
params = utils.Struct(**args)
params.generic.device = DEVICE
args

In [ ]:
train_dataset, test_dataset = train.get_dataset(params)

In [ ]:
interact(
    draw_targets, 
    dataset=fixed(train_dataset), 
    index=widgets.IntSlider(min=0, max=len(train_dataset), step=1, value=10)
);

## Training

In [ ]:
train_dataloader = train.get_train_dataloader(params, train_dataset)
test_dataloader = train.get_test_dataloader(params, test_dataset)

In [ ]:
detector = detectors.get_detector(params, train.NUM_CLASSES)
detector.to(DEVICE)

In [ ]:
train.wandb_init(params, args)
train.wandb_watch(params, detector)

In [ ]:
optimizer = train.get_optimizer(params, detector)
lr_scheduler = train.get_lr_scheduler(params, optimizer)

In [ ]:
learning.training_loop(
    params, detector, optimizer, train_dataloader,
    test_dataloader, lr_scheduler=lr_scheduler
)

In [ ]:
train.wandb_finish()

## Testing

In [ ]:
detector.eval()

In [ ]:
interact(
    draw_predictions, 
    dataset=fixed(test_dataset),
    index=widgets.IntSlider(min=0, max=len(test_dataset), step=1, value=10),
    model=fixed(detector),
    conf_thresh=widgets.FloatSlider(min=0.0, max=1.0, step=0.05, value=0.5)
);